# Import libraries

In [1]:
import pandas as pd
import numpy as np


In [2]:
import pymysql
from sqlalchemy import create_engine
import getpass 
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/clean_winery'
engine = create_engine(connection_string)

In [4]:
query = '''SELECT wine_id, type, wine_name, country, varietal_name, IFNULL(style_id, 0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM wine
LEFT JOIN purchase USING (wine_id)
LEFT JOIN  style USING (wine_id)
ORDER BY style_id;'''
df = pd.read_sql(query,engine)

In [5]:
df.shape

(20467, 13)

In [6]:
varietal_names = df['varietal_name'].unique()

In [7]:
len(varietal_names)

104

In [8]:
varietal_names = np.delete(varietal_names,0)
len(varietal_names)

103

In [9]:
'Blanco' in varietal_names

True

## Clean varietal names

In [10]:
to_clean = df[df['style_id']==0]

In [11]:
to_clean

,wine_id,type,wine_name,country,varietal_name,style_id,body,acidity_1,acidity_2,fizziness,intensity,sweetness,tannin
0,170766130,2,Cala Marquesa Garnacha Blanca,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,162466975,2,Dominio do Bibei Refu,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,168714635,2,Naia K-Naia,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,158943250,2,Muchada-Léclapart Univers,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,165941865,2,Oller del Mas Bernat Oller Blanc de Picapolls,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,168580580,2,Reboreda Adeus,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2171,169102057,2,Merayo Godello Bierzo,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2172,171055783,2,Castellroig Finca Sabaté i Coca Sø Blanco,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2173,170625872,2,De Moya Tibó Blanco,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
to_clean.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

2175

In [13]:
query_godello = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Godello') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Godello%%');'''
godello = pd.read_sql(query_godello,engine)

In [14]:
godellos_id = godello['wine_id'].values
godello.shape

(105, 13)

In [15]:
query_garnacha = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Grenache.') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Garnacha%%') OR (wine_name LIKE '%%Garnatxa%%') OR (wine_name LIKE '%%Grenache%%');'''
garnacha = pd.read_sql(query_garnacha,engine)

In [16]:
garnacha_id = garnacha['wine_id'].values
garnacha.shape

(80, 13)

In [17]:
query_moscatel_ch = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Moscatel - Chardonnay') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Chardonnay - Moscatel%%') OR (wine_name LIKE '%%Moscatel - Chardonnay%%');'''
moscatel_ch = pd.read_sql(query_moscatel_ch,engine)

In [18]:
mosc_ch_id = moscatel_ch['wine_id'].values
moscatel_ch.shape

(2, 13)

In [19]:
query_moscatel = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Moscatel') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Moscatel%%') OR (wine_name LIKE '%%Muscadel%%');'''
moscatel = pd.read_sql(query_moscatel,engine)

In [20]:
moscatel.shape

(20, 13)

In [21]:
moscatel = moscatel[moscatel['wine_id'].isin(mosc_ch_id)==False]
moscatel_id = moscatel['wine_id'].values
moscatel.shape

(18, 13)

In [22]:
query_treixadura = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Treixadura') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Treixadura%%';'''
treixadura = pd.read_sql(query_treixadura,engine)

In [23]:
treixadura_id = treixadura['wine_id'].values
treixadura.shape

(12, 13)

In [24]:
query_muscat_sav = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Sauvignon Blanc - Muscat') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Sauvignon Blanc - Muscat%%';'''
muscat_sav = pd.read_sql(query_muscat_sav,engine)

In [25]:
muscat_sav_id = muscat_sav['wine_id'].values
muscat_sav

,wine_id,type,wine_name,country,varietal_name,style_id,body,acidity_1,acidity_2,fizziness,intensity,sweetness,tannin
0,170588290,2,Emendis Flos Sauvignon Blanc - Muscat,spain,Sauvignon Blanc - Muscat,0,None,None,None,None,None,None,None
1,163309828,2,Emendis Flos Sauvignon Blanc - Muscat,spain,Sauvignon Blanc - Muscat,0,None,None,None,None,None,None,None


In [26]:
query_muscat = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Muscat') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Muscat%%';'''
muscat = pd.read_sql(query_muscat,engine)

In [27]:
muscat.shape

(16, 13)

In [28]:
muscat = muscat[muscat['wine_id'].isin(muscat_sav_id)==False]
muscat_id = muscat['wine_id'].values
muscat.shape

(14, 13)

In [29]:
query_sauv = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Sauvignon Blanc') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Sauvignon Blanc%%';'''
sauv = pd.read_sql(query_sauv,engine)

In [30]:
sauv.shape

(35, 13)

In [31]:
sauv = sauv[sauv['wine_id'].isin(muscat_sav_id)==False]
sauv_id = sauv['wine_id'].values
sauv.shape

(33, 13)

In [32]:
query_albariño = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'ALbariño') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Albariño%%';'''
albariño = pd.read_sql(query_albariño,engine)

In [33]:
albariño_id = albariño['wine_id'].values
albariño.shape

(3, 13)

In [34]:
query_cabernet_f = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Cabernet Franc') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Cabernet Franc%%';'''
cabernet_f = pd.read_sql(query_cabernet_f,engine)

In [35]:
cabernet_f_id = cabernet_f['wine_id'].values
cabernet_f.shape

(23, 13)

In [36]:
query_cabernet_s = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Cabernet Sauvignon') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Cabernet Sauvignon%%';'''
cabernet_s = pd.read_sql(query_cabernet_s,engine)

In [37]:
cabernet_s_id = cabernet_s['wine_id'].values
cabernet_s.shape

(7, 13)

In [38]:
query_cabernet_sh = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Cabernet - Shiraz') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Cabernet - Shiraz%%';'''
cabernet_sh = pd.read_sql(query_cabernet_sh,engine)

In [39]:
cabernet_sh_id = cabernet_sh['wine_id'].values
cabernet_sh.shape

(1, 13)

In [40]:
def ids_clean(df,values):
    for value in values:
        df = df[df['wine_id'].isin(value)==False]
    return df

In [41]:
values = [muscat_id,cabernet_s_id,sauv_id, muscat_sav_id, godellos_id,garnacha_id,mosc_ch_id,moscatel_id,treixadura_id,cabernet_sh_id,cabernet_f_id,albariño_id]

In [42]:
to_clean_2 = ids_clean(to_clean,values)

In [43]:
to_clean_2

,wine_id,type,wine_name,country,varietal_name,style_id,body,acidity_1,acidity_2,fizziness,intensity,sweetness,tannin
1,162466975,2,Dominio do Bibei Refu,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,168714635,2,Naia K-Naia,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,158943250,2,Muchada-Léclapart Univers,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,165941865,2,Oller del Mas Bernat Oller Blanc de Picapolls,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,168620066,2,Mas d'en Gil Bellmunt Vi de Villa Blanc,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,152298281,2,Clos Pons Sisquella,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2170,168580580,2,Reboreda Adeus,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2172,171055783,2,Castellroig Finca Sabaté i Coca Sø Blanco,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2173,170625872,2,De Moya Tibó Blanco,spain,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
to_clean_2.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

1877

In [45]:
query_chardonnay_pn = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Chardonnay - Pinot Noir') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Chardonnay - Pinot Noir%%';'''
chardonnay_pn = pd.read_sql(query_chardonnay_pn,engine)

In [46]:
chardonnay_pn_id = chardonnay_pn['wine_id'].values
chardonnay_pn.shape

(3, 13)

In [47]:
query_pinot_noir = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Pinot Noir') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Pinot Noir%%';'''
pinot_noir = pd.read_sql(query_pinot_noir,engine)

In [48]:
pinot_noir = pinot_noir[pinot_noir['wine_id'].isin(chardonnay_pn_id)==False]
pinot_noir_id = pinot_noir['wine_id'].values
pinot_noir.shape

(4, 13)

In [49]:
query_chardonnay = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Chardonnay') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Chardonnay%%';'''
chardonnay = pd.read_sql(query_chardonnay,engine)

In [50]:
chardonnay = chardonnay[chardonnay['wine_id'].isin(chardonnay_pn_id)==False]
chardonnay_id = chardonnay['wine_id'].values
chardonnay.shape

(25, 13)

In [51]:
query_ribeiro = '''SELECT wine_id, w.type, w.wine_name, w.country, IFNULL(varietal_name, 'Ribeiro') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM wine w
JOIN to_clean USING (wine_id)
WHERE region LIKE '%%Ribeiro%%';'''
ribeiro = pd.read_sql(query_ribeiro,engine)

In [52]:
ribeiro_id = ribeiro['wine_id'].values
ribeiro.shape

(53, 13)

In [53]:
query_verdelho = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Verdejo') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Verdelho%%') OR (wine_name LIKE '%%Verdejo%%');'''
verdelho = pd.read_sql(query_verdelho,engine)

In [54]:
verdelho_id = verdelho['wine_id'].values
verdelho.shape

(2, 13)

In [55]:
values_2 = [chardonnay_pn_id, pinot_noir_id,chardonnay_id,ribeiro_id,verdelho_id]
to_clean_2 = ids_clean(to_clean_2,values_2)
to_clean_2.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

1790

In [56]:
query_barbera = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Barbera') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Barbera%%';'''
barbera = pd.read_sql(query_barbera,engine)

In [57]:
barbera_id = barbera['wine_id'].values
barbera.shape

(4, 13)

In [58]:
query_ch_vi = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Chenin blanc - Viognier') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Chenin blanc - Viognier%%';'''
ch_vi = pd.read_sql(query_ch_vi,engine)

In [59]:
ch_vi_id = ch_vi['wine_id'].values
ch_vi.shape

(1, 13)

In [60]:
query_chenin = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Chenin blanc') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Chenin blanc%%';'''
chenin = pd.read_sql(query_chenin,engine)

In [61]:
chenin = chenin[chenin['wine_id'].isin(ch_vi_id)==False]
chenin_id = chenin['wine_id'].values
chenin.shape

(7, 13)

In [62]:
query_geu = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Gewürztraminer') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Gewürztraminer%%';'''
geu = pd.read_sql(query_geu,engine)

In [63]:
geu_id = geu['wine_id'].values
geu.shape

(23, 13)

In [64]:
query_m_s = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Malbec - Syrah') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Malbec - Syrah%%';'''
mal_sy = pd.read_sql(query_m_s,engine)

In [65]:
mal_sy_id = mal_sy['wine_id'].values
mal_sy.shape

(2, 13)

In [66]:
query_malbec = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Malbec') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Malbec%%';'''
malbec = pd.read_sql(query_malbec,engine)

In [67]:
malbec = malbec[malbec['wine_id'].isin(mal_sy_id)==False]
malbec_id = malbec['wine_id'].values
malbec.shape

(4, 13)

In [68]:
query_mencia = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Mencía') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Mencía%%';'''
mencia = pd.read_sql(query_mencia,engine)

In [69]:
mencia_id = mencia['wine_id'].values
mencia.shape

(2, 13)

In [70]:
query_mer_s = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Merlot - Syrah') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Merlot - Syrah%%';'''
mer_sy = pd.read_sql(query_mer_s,engine)

In [71]:
mer_sy_id = mer_sy['wine_id'].values
mer_sy.shape

(1, 13)

In [72]:
query_merlot = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Merlot') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Merlot%%';'''
merlot = pd.read_sql(query_merlot,engine)

In [73]:
merlot = merlot[merlot['wine_id'].isin(mer_sy_id)==False]
merlot_id = merlot['wine_id'].values
merlot.shape

(9, 13)

In [74]:
query_nebiolo = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Nebbiolo') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Nebbiolo%%';'''
nebiolo = pd.read_sql(query_nebiolo,engine)

In [75]:
nebiolo_id = nebiolo['wine_id'].values
nebiolo.shape

(4, 13)

In [76]:
query_pe_si = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Petite Sirah') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Petite Sirah%%';'''
pe_si = pd.read_sql(query_pe_si,engine)

In [77]:
pe_si_id = pe_si['wine_id'].values
pe_si.shape

(2, 13)

In [78]:
query_pi_bl = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Pinot Blanc') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Pinot Blanc%%';'''
pinot_blanc = pd.read_sql(query_pi_bl,engine)

In [79]:
pi_bl_id = pinot_blanc['wine_id'].values
pinot_blanc.shape

(2, 13)

In [80]:
query_spa = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Spätburgunder') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Spätburgunder%%';'''
spa = pd.read_sql(query_spa,engine)

In [81]:
spa_id = spa['wine_id'].values
spa.shape

(1, 13)

In [82]:
query_pinotage = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Pinotage') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Pinotage%%';'''
pinotage = pd.read_sql(query_pinotage,engine)

In [83]:
pinotage_id = pinotage['wine_id'].values
pinotage.shape

(1, 13)

In [84]:
query_blend_t = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Blend tinto') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Blend tinto%%';'''
blend_t = pd.read_sql(query_blend_t,engine)

In [85]:
blend_t_id = blend_t['wine_id'].values
blend_t.shape

(2, 13)

In [86]:
query_sangio = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Sangiovese') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Sangiovese%%';'''
sangio = pd.read_sql(query_sangio,engine)

In [87]:
sangio_id = sangio['wine_id'].values
sangio.shape

(5, 13)

In [88]:
query_syrah = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Syrah') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Syrah%%';'''
syrah = pd.read_sql(query_syrah,engine)

In [89]:
syrah = syrah[syrah['wine_id'].isin(mer_sy_id)==False]
syrah_id = syrah['wine_id'].values
syrah.shape

(19, 13)

In [90]:
query_vio = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Viognier') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Viognier%%';'''
vio = pd.read_sql(query_vio,engine)

In [91]:
vio = vio[vio['wine_id'].isin(ch_vi_id)==False]
vio_id = vio['wine_id'].values
vio.shape

(13, 13)

In [92]:
query_zin = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Zinfandel') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Zinfandel%%';'''
zin = pd.read_sql(query_zin,engine)

In [93]:
zin_id = zin['wine_id'].values
zin.shape

(1, 13)

In [94]:
query_prim = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Primitivo') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Primitivo%%';'''
prim = pd.read_sql(query_prim,engine)

In [95]:
prim_id = prim['wine_id'].values
prim.shape

(3, 13)

In [96]:
query_tor_ries = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Torrontés - Riesling') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Torrontés - Riesling%%';'''
tor_ries = pd.read_sql(query_tor_ries,engine)

In [97]:
tor_ries_id = tor_ries['wine_id'].values
tor_ries.shape

(2, 13)

In [98]:
query_riesling = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Riesling') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Riesling%%';'''
riesling = pd.read_sql(query_riesling,engine)

In [99]:
riesling = riesling[riesling['wine_id'].isin(tor_ries_id)==False]
riesling_id = riesling['wine_id'].values
riesling.shape

(11, 13)

In [100]:
query_tempranillo = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Tempranillo') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE wine_name LIKE '%%Tempranillo%%';'''
tempranillo = pd.read_sql(query_tempranillo,engine)

In [101]:
tempranillo_id = tempranillo['wine_id'].values
tempranillo.shape

(3, 13)

In [102]:
for i in range(len(varietal_names)):
    print(varietal_names[i])

Albariño
Tinto de Alentejo
Amarone
Barbaresco
Barbera
Barolo
Tinto
Bolgheri
Bonarda
Blend de Burdeos
Blanco
Brunello
Cabernet Franc
Cabernet - Malbec
Cabernet Sauvignon
Cabernet - Shiraz
Cabernet - Syrah
Carménère
Chablis
Chardonnay
Chenin blanc
Chianti
Côte-Rotie
Tinto de Dão
Dornfelder
Tinto de Douro
Gavi
Gewürztraminer
Grenache.
Cannonau
Malbec
Mencía
Merlot
Monastrell
Montepulciano d'Abruzzo
Tinto de Montsant
Moscatel
Müller Thurgau
Nebbiolo
Pedro Ximenez
Petite Sirah
Pinot Blanc
Pinot Gris
Grauburgunder
Pinot gris
Pinot noir
Spätburgunder
Blend de Pinotage
Pinotage
Tinto de Priorat
Blend tinto
Blend del Ródano
Tinto de Ribera del Duero
Riesling
Tinto de Rioja
Blanco de Rioja
Ripasso
Sangiovese
Sauvignon blanc
Jerez
Soave
Vino espumoso
Syrah
Shiraz
Syrah - Viognier
Tinto de Toro
Tinto de Valpolicella
Verdejo
Blanco vinho verde
Vino Nobile di Montepulciano
Viognier
Blend blanco
Zinfandel
Primitivo
Silvaner
Tinto de Graves
Pessac-Léognan
Médoc
Margaux
Pauillac
Saint-Julien
Saint-Émil

In [103]:
values_3 = [barbera_id,ch_vi_id,chenin_id,geu_id,mal_sy_id,malbec_id,mencia_id,mer_sy_id,merlot_id,nebiolo_id,pe_si_id,pi_bl_id,spa_id,pinotage_id,tempranillo_id,riesling_id,tor_ries_id,prim_id,zin_id,vio_id,syrah_id,sangio_id,blend_t_id]
to_clean_2 = ids_clean(to_clean_2,values_3)
to_clean_2.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

1671

In [104]:
query_blanco = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Blanco') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Blanco%%') OR (wine_name LIKE '%%Blanc%%') OR (wine_name LIKE '%%White%%')'''
blanco = pd.read_sql(query_blanco,engine)

In [105]:
blanco_id = blanco['wine_id'].values
blanco.shape

(502, 13)

In [106]:
values_4 = [blanco_id]
to_clean_2 = ids_clean(to_clean_2,values_4)
to_clean_2.to_sql('to_clean_2', con = engine, if_exists = 'replace', index = False )

1169

In [107]:
query_tinto = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Tinto') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE (wine_name LIKE '%%Tinto%%') OR (wine_name LIKE '%%Red%%') OR (wine_name LIKE '%%Rouge%%')'''
tinto = pd.read_sql(query_tinto,engine)

In [108]:
tinto_id = tinto['wine_id'].values
tinto.shape

(152, 13)

In [109]:
values_5 = [tinto_id]
to_clean_2 = ids_clean(to_clean_2,values_5)
to_clean_2.to_sql('to_clean_2', con = engine, if_exists = 'replace', index = False )

1020

In [110]:
query_tinto_2 = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Tinto') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE type = 1'''
tinto_2 = pd.read_sql(query_tinto_2,engine)

In [111]:
tinto_2_id = tinto_2['wine_id'].values
tinto_2.shape

(551, 13)

In [112]:
query_blanco_2 = '''SELECT wine_id, type, wine_name, country, IFNULL(varietal_name, 'Blanco') AS varietal_name, IFNULL(style_id,0) AS style_id, body, acidity_1, acidity_2, fizziness, intensity, sweetness, tannin FROM to_clean
WHERE type = 2'''
blanco_2 = pd.read_sql(query_blanco_2,engine)

In [113]:
blanco_2_id = blanco_2['wine_id'].values
blanco_2.shape

(1120, 13)

In [114]:
values_6 = [tinto_2_id,blanco_2_id]
to_clean_2 = ids_clean(to_clean_2,values_6)
to_clean_2.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

0

## Get together all the cleaned data

In [115]:
cleaned_varietal_names = pd.concat([godello,garnacha,moscatel_ch,moscatel,treixadura,muscat_sav,muscat,sauv,albariño,cabernet_f,cabernet_s,cabernet_sh,chardonnay_pn,pinot_noir,chardonnay,ribeiro,verdelho,barbera,ch_vi,chenin,geu,mal_sy,malbec,mencia,mer_sy,merlot,nebiolo,pe_si,pinot_blanc,spa,pinotage,blend_t,sangio,syrah,vio,zin,prim,tor_ries,riesling,tempranillo,blanco,tinto,blanco_2,tinto_2],axis=0)
cleaned_varietal_names.shape

(2834, 13)

In [116]:
ready = df[df['style_id']!=0]

In [117]:
new_df = pd.concat([ready,cleaned_varietal_names],axis=0)

In [118]:
print(df.shape)
print(new_df.shape)

(20467, 13)
(21126, 13)


## Check duplicates that I created

In [119]:
new_df.duplicated(subset=['wine_id']).sum()

659

In [120]:
duplicated_id = new_df[new_df['wine_id'].duplicated()==True]['wine_id'].values

In [121]:
new_df[new_df['wine_id'].isin(duplicated_id)].sort_values(by='wine_id')

,wine_id,type,wine_name,country,varietal_name,style_id,body,acidity_1,acidity_2,fizziness,intensity,sweetness,tannin
264,1295802,1,Bucaco Tinto Reservado,portugal,Tinto,0,None,None,None,None,None,None,None
79,1295802,1,Bucaco Tinto Reservado,portugal,Tinto,0,None,None,None,None,None,None,None
234,1322646,1,Clos Rougeard Les Poyeux Saumur Champigny,france,Tinto,0,None,None,None,None,None,None,None
71,1322646,1,Clos Rougeard Les Poyeux Saumur Champigny,france,Tinto,0,None,None,None,None,None,None,None
126,1399141,1,Bucaco Tinto Reservado,portugal,Tinto,0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,172025160,2,Clos Figueras Font de la Figuera Priorat Blanco,spain,Blanco,0,None,None,None,None,None,None,None
82,172098560,2,Serra i Barceló Aucalà Blanc,spain,Blanco,0,None,None,None,None,None,None,None
170,172098560,2,Serra i Barceló Aucalà Blanc,spain,Blanco,0,None,None,None,None,None,None,None
329,172210226,2,Telmo Rodriguez O Barreiro Gaba do Xil Blanco,spain,Blanco,0,None,None,None,None,None,None,None


In [122]:
new_df.shape

(21126, 13)

In [123]:
len(new_df[new_df['varietal_name'] == 'Grenache.'])

158

In [124]:
len(new_df[new_df['varietal_name'] == 'Moscatel'])

20

In [125]:
len(new_df[new_df['varietal_name'] == 'Sauvignon Blanc'])

33

In [126]:
len(new_df[new_df['varietal_name'] == 'Riesling'])

975

In [127]:
len(new_df[new_df['varietal_name'] == 'Gewürztraminer'])

117

In [128]:
new_df2 = new_df[~((new_df['wine_id'] == 156769689) & (new_df['varietal_name'] == 'Syrah'))]
new_df2 = new_df2[~((new_df2['wine_id'] == 163986916) & (new_df2['varietal_name'] == 'Syrah'))]
new_df2 = new_df2[~((new_df2['wine_id'] == 167545383) & (new_df2['varietal_name'] == 'Moscatel'))]
new_df2 = new_df2[~((new_df2['wine_id'] == 167919003) & (new_df2['varietal_name'] == 'Gewürztraminer'))]
new_df2 = new_df2[~((new_df2['wine_id'] == 171437200) & (new_df2['varietal_name'] == 'Sauvignon Blanc'))]
new_df2.shape

(21121, 13)

In [129]:
print(df.shape)
print(new_df2.shape)

(20467, 13)
(21121, 13)


In [130]:
new_df2.to_csv('to_clean.csv',index = False)
new_df2.to_sql('to_clean', con = engine, if_exists = 'replace', index = False )

21121